In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg
import numpy as np
import galois   

In [2]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [3]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print(1,out,x,y,v1,v2,v3,v4)

Lgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

Rgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

Ogf = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

wgf = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = Ogf.dot(wgf) == np.multiply(Lgf.dot(wgf),Rgf.dot(wgf))
assert result.all(), "result contains an inequality"


1 97 5 10 25 125 100 1250


In [4]:
target_poly = GF(np.array([1,2,3,4,5]))
# for each column in the matrix, compute the lagrange polynomial, resulting in a vector of polynomials
def interpolate_columns(column):
    return galois.lagrange_poly(target_poly,column)

L_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Lgf)
R_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Rgf)
O_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Ogf)
print(R_vector_of_poly)

[Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(36480404786398792037077342908762125147580607334

In [31]:
def reduce_polynomials_to_1(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

Ua = reduce_polynomials_to_1(L_vector_of_poly,wgf)
Va = reduce_polynomials_to_1(R_vector_of_poly,wgf)
Wa = reduce_polynomials_to_1(O_vector_of_poly,wgf)
# print(Ua)
print(Va)
# print(Wa)

13680151794899547013904003590785796930342727750260021464811377616609880309765x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539795x^3 + 8208091076939728208342402154471478158205636650156012878886826569965928186022x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955532x + 125


In [70]:
# Compute Generator points and add together
# Uses py_ecc to scale both G1 and G2 points
from functools import reduce
import random

tau = GF(9)
# tau = GF(random.randint(1,curve_order-1))

def generate_powers_of_tau_G1(tau, degree):
    """
    tau is a random num between 1 and curve_order, degree is the highest d of the polynomial
    this generates the powers of tau, i.e. tau^0, tau^1, tau^2, tau^3, tau^4, tau^5
    """
    return [multiply(G1, int(tau ** i)) for i in range(degree + 1)]

def generate_powers_of_tau_G2(tau, degree):
    """
    same for G2
    """
    return [multiply(G2, int(tau ** i)) for i in range(degree + 1)]

def inner_product(ec_points, coeffs):
    """
    ec_points is the powers of tao, i.e. [G1, tau*G1, tau^2*G1, tau^3*G1, ...]
    coeffs are the coefficients of the polynomial after lagrange
    this computes the inner product of the 2 vectors i.e. Ux[0] * tau^0 + Ux[1] + tau^1..
    the result is 1 point on the curve representing the whole polynomial
    Z1 is the identity element
    """
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

# generate target polynomial (x-1)(x-2)... 
target_poly_x = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])).astype(int),  field=GF)

# Public Trusted Setup for powers of tau G1, G2
powers_of_tau_G1 = generate_powers_of_tau_G1(tau, 2 * target_poly_x.degree)
powers_of_tau_G2 = generate_powers_of_tau_G2(tau, 2 * target_poly_x.degree)

# inner product of R1CS matrix and Witness Vector, computed by Prover
inner_product_ua_and_tau = inner_product(powers_of_tau_G1, Ua.coeffs[::-1])
inner_product_va_and_tau = inner_product(powers_of_tau_G2, Va.coeffs[::-1])
inner_product_wa_and_tau = inner_product(powers_of_tau_G1, Wa.coeffs[::-1])

# [A][B] = [C] + h(x)t(x)
A1 = inner_product_ua_and_tau
B2 = inner_product_va_and_tau

# h(x) = Ua * Va - Wa // target_poly_x computed by prover
hx, remainder = divmod((Ua * Va - Wa), target_poly_x)
assert remainder == 0, "remainder is not 0"

ht = hx * target_poly_x
inner_product_ht_and_tau = inner_product(powers_of_tau_G1, ht.coeffs[::-1])
C1 = add(inner_product_wa_and_tau, inner_product_ht_and_tau)
print(C1)

if(eq(pairing(B2,A1), pairing(G2,C1))):
    print("true")
else:
    print("false")


(15408243139265932337676760254044817568949149914733382727758642401124772145172, 10707466251951667763649266781350499983831580247483887965971176433260431209127)
true


In [74]:
# Trusted Setup
alpha = 2
beta = 3
alpha1 = multiply(G1, alpha)
beta2 = multiply(G2, beta)


def generate_powers_of_tau_c(tau, degree):
    return [int(tau ** i) for i in range(degree)]

powers_of_tau = generate_powers_of_tau_c(tau,len(wgf))

def reduce_polynomials_to_1(vector_of_poly, powers_of_tau):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i] * powers_of_tau[i]
    return sum

Utau = reduce_polynomials_to_1(L_vector_of_poly, powers_of_tau)
Vtau = reduce_polynomials_to_1(R_vector_of_poly, powers_of_tau)
Wtau = reduce_polynomials_to_1(O_vector_of_poly, powers_of_tau)
print(Utau)

beta_utau = beta * Utau
alpha_vtau = alpha * Vtau

combined_powers_of_tau_for_c = beta_utau + alpha_vtau + Wtau

# evaluate at tau with inner product and G1
inner_product_powers_of_tau_for_c = inner_product(powers_of_tau_G1, combined_powers_of_tau_for_c.coeffs[::-1])
print(inner_product_powers_of_tau_for_c)

# target polynomial evaluated at tau
target_poly_tau_G1 = inner_product(powers_of_tau_G1,target_poly_x.coeffs[::-1])
print(target_poly_tau_G1)





# powers of tau for h(tau)t(tau)
# t(tau) = target polynomial evaluated at tau

5472060717959818805561601436314318772137091100104008585924551046643952113138x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904368215x^3 + 16416182153879456416684804308942956316411273300312025757773653139931855912274x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904951253x + 21888242871839275222246405745257275088548364400416034343698204186575808142052
(9326956671834866868468699811816534810103024654683332412025578094365989586332, 4331078678490706627067589576256765366988401669845479238880291071712418357445)
(10058941465250107733086766383700277733512738938144443928941160957396381143837, 8819704791670307923985668120646765434900169523120333769019554546763301791772)
